## DSCI 100 Project
### Individual Planning Stage

### Part 1 - Data Description

The dataset consists of two comma-separated values files, `players.csv` and `sessions.csv`, both collected from stats of players in the game. The data seems to be taken from game logs, with user-inputted information from players profiles serving as the different variables. User-inputted information is not always 100% accurate, as some users may falsify their Age, gender, or name.

#### `players.csv`
- 196 observations and 7 variables
| Variable      | Type     | Meaning                                                      |
|---------------|----------|--------------------------------------------------------------|
| experience    | Character | Describes the experience level of the player.               |
| subscribe     | Logical   | "TRUE" if players are subscribed, "FALSE" if not subscribed.|
| hashedEmail   | Character | Unique ID code for each player.                             |
| played_hours  | Double    | Number of hours spent in-game, as a decimal.                |
| name          | Character | The name of each player.                                    |
| gender        | Character | The gender of each player.                                  |
| Age           | Double    | Age of each player.                                         |


- Of 196 total players, 144 are subscribed and 52 are not subscribed
- In the experience variable, there are 63 Amateur, 35 Beginner, 14 Pro, 36 Regular, and 48 Veteran
- A summary of the quantitative variable `played_hours`, grouped by experience level:
| experience  | min | max   | mean       |
|-------------|-----|-------|------------|
| All         | 0   | 223.10|   5.85     |
| Amateur     | 0   | 150.00| 6.02       |
| Beginner    | 0   | 23.70 | 1.25       |
| Pro         | 0   | 30.30 | 2.60       |
| Regular     | 0   | 223.10| 18.21      |
| Veteran     | 0   | 12.50 | 0.65       |

- A summary of the quantitative variable `Age`, grouped by gender (Other and Two-Spirited exlcuded, as no Age was entered):
| gender             | min | max | mean     |
|--------------------|-----|-----|----------|
| All                | 9   |  58 | 21.14    |
| Agender            | 20  | 27  | 23.50    |
| Female             | 16  | 44  | 21.81    |
| Male               | 9   | 58  | 20.85    |
| Non-binary         | 17  | 25  | 19.07    |
| Prefer not to say  | 17  | 57  | 25.00    | 

##### Issues
43% of the dataset logs 0 hours played, which could potential skew the data.

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------